In [1]:
import os, sys
sys.path.insert(0, '/home/trax/trax/website')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "trax.settings")
import django
django.setup()

In [67]:
from fuzzywuzzy import fuzz, process
from tracks.models import Track
from vehicles.models import Vehicle, VehicleClass
from trax.utils import get_object_by_string

def lt2millis(parts):
    m, rest = parts.split(':')
    millis = 1000 * (int(m)*60 + float(rest))
    return millis

def get_object_by_string(s, klass, attribute, score_cutoff=90):
    choices = klass.objects.all().values_list(attribute, flat=True)
    result = process.extractOne(s, choices,
                                score_cutoff=score_cutoff)
    if result:
        return klass.objects.get(**{attribute:result[0]})


In [3]:
data = '''Motorcycles	Akuma	0:58.892	121.25
Motorcycles	Avarus	1:07.168	107.5
Motorcycles	Bagger	1:13.907	98
Motorcycles	Bati 801	0:58.625	131.5
Motorcycles	BF400	1:03.263	127
Motorcycles	Carbon RS	1:00.960	124.25
Motorcycles	Chimera	1:08.935	103
Motorcycles	Cliffhanger	1:02.595	124.75
Motorcycles	Daemon	1:08.635	106.5
Motorcycles	Daemon (Lost)	1:07.834	107
Motorcycles	Defiler	1:00.326	117
Motorcycles	Diabolus	1:04.297	114.25
Motorcycles	Diabolus Custom	1:02.896	115.25
Motorcycles	Double-T	1:00.261	118
Motorcycles	Enduro	1:09.137	107.25
Motorcycles	Esskey	1:06.399	112.75
Motorcycles	Faggio	1:30.125	103.25
Motorcycles	Faggio Mod	1:27.154	74.25
Motorcycles	Faggio Sport	1:25.318	76.75
Motorcycles	FCR 1000	1:03.230	112
Motorcycles	FCR 1000 Custom	1:02.396	112.5
Motorcycles	Gargoyle	1:02.628	125
Motorcycles	Hakuchou	1:00.760	131
Motorcycles	Hakuchou Drag	0:57.590	126.5
Motorcycles	Hexer	1:07.167	104.75
Motorcycles	Innovation	1:04.999	110
Motorcycles	Lectro	1:03.729	109.25
Motorcycles	Manchez	1:06.465	113.5
Motorcycles	Nemesis	1:04.598	111.25
Motorcycles	Nightblade	1:03.496	114.25
Motorcycles	PCJ 600	1:06.734	106.75
Motorcycles	Rat Bike	1:17.632	123.5
Motorcycles	Ruffian	1:02.461	122.25
Motorcycles	Sanchez	1:04.532	117.75
Motorcycles	Sanctus	1:03.329	112
Motorcycles	Shotaro	0:57.490	123.75
Motorcycles	Sovereign	1:06.400	106.25
Motorcycles	Thrust	1:07.267	118.25
Motorcycles	Vader	1:05.299	107.75
Motorcycles	Vindicator	1:05.466	121.25
Motorcycles	Vortex	1:01.028	115.75
Motorcycles	Wolfsbane	1:17.444	101.5
Motorcycles	Zombie Bobber	1:08.311	113.75
Motorcycles	Zombie Chopper	1:08.201	113.75
Muscle	Blade	1:09.269	109.75
Muscle	Buccaneer	1:12.239	113
Muscle	Buccaneer Custom	1:12.221	112.75
Muscle	Burger Shot Stallion	1:10.904	113.25
Muscle	Chino	1:19.946	94.25
Muscle	Chino Custom	1:19.879	95.5
Muscle	Coquette BlackFin	1:09.803	114.25
Muscle	Dominator	1:09.035	120.25
Muscle	Duke O'Death	1:10.104	114.25
Muscle	Dukes	1:09.636	112.5
Muscle	Faction	1:16.476	110.75
Muscle	Faction Custom	1:13.372	110.75
Muscle	Faction Custom Donk	1:16.275	97.5
Muscle	Gauntlet	1:12.005	113.25
Muscle	Hotknife	1:19.079	109.5
Muscle	Lost Slamvan	1:24.981	95.25
Muscle	Lurcher	1:14.406	114.25
Muscle	Moonbeam	1:21.647	102
Muscle	Moonbeam Custom	1:17.477	102
Muscle	Nightshade	1:14.841	104.5
Muscle	Phoenix	1:11.172	113
Muscle	Picador	1:17.878	102.5
Muscle	Pisswasser Dominator	1:06.766	124.5
Muscle	Rat-Loader	1:24.785	102.5
Muscle	Rat-Truck	1:24.084	105.25
Muscle	Redwood Gauntlet	1:11.471	117.25
Muscle	Ruiner	1:09.936	118.75
Muscle	Ruiner 2000	1:10.063	118.75
Muscle	Sabre Turbo	1:10.136	110.75
Muscle	Sabre Turbo Custom	1:06.866	121.25
Muscle	Slamvan	1:25.451	108
Muscle	Slamvan Custom	1:11.005	117.5
Muscle	Stallion	1:11.572	110.25
Muscle	Tampa	1:11.170	105.5
Muscle	Vigero	1:13.673	112
Muscle	Virgo	1:18.912	97.5
Muscle	Virgo Classic	1:21.448	96.75
Muscle	Virgo Classic Custom	1:15.441	104
Muscle	Voodoo	1:22.516	99
Muscle	Voodoo Custom	1:22.181	100.75
Sports	9F	1:04.125	119.75
Sports	9F Cabrio	1:04.157	119.75
Sports	Alpha	1:05.732	117.75
Sports	Banshee	1:06.066	117.75
Sports	Bestia GTS	1:04.364	118.75
Sports	Blista Compact	1:15.408	103
Sports	Buffalo	1:08.503	112.25
Sports	Buffalo S	1:08.001	112.25
Sports	Carbonizzare	1:05.699	119.5
Sports	Comet	1:05.130	119.5
Sports	Comet Retro Custom	1:05.632	121.25
Sports	Coquette	1:06.333	119.25
Sports	Drift Tampa	1:08.334	114.25
Sports	Elegy Retro Custom	1:03.483	115.25
Sports	Elegy RH8	1:02.529	118.5
Sports	Feltzer	1:03.229	119.5
Sports	Furore GT	1:08.502	120.25
Sports	Fusilade	1:08.652	117.75
Sports	Futo	1:12.706	119.25
Sports	Go Go Monkey Blista	1:15.402	103
Sports	Jester	1:03.763	118.75
Sports	Jester (Racecar)	1:03.363	119.25
Sports	Khamelion	1:08.301	102.25
Sports	Kuruma	1:06.800	112
Sports	Kuruma (Armored)	1:09.036	109.75
Sports	Lynx	1:07.133	121.5
Sports	Massacro	1:03.864	121.75
Sports	Massacro (Racecar)	1:03.810	121.75
Sports	Omnis	1:07.600	112.5
Sports	Penumbra	1:12.243	105.25
Sports	Rapid GT	1:08.419	119.5
Sports	Rapid GT	1:08.434	119.5
Sports	Raptor	1:11.371	103.25
Sports	Schafter LWB	1:09.769	109.5
Sports	Schafter V12	1:05.121	124.25
Sports	Schwartzer	1:11.105	117
Sports	Seven-70	1:04.764	123.5
Sports	Specter	1:04.398	121.25
Sports	Specter Custom	1:04.998	121.25
Sports	Sprunk Buffalo	1:07.234	116
Sports	Sultan	1:05.834	115.75
Sports	Surano	1:04.531	121
Sports	Tropos Rallye	1:06.866	119.5
Sports	Verlierer	1:05.659	121.75
Sports Classics	Casco	1:10.337	120
Sports Classics	Coquette Classic	1:11.470	118
Sports Classics	Franken Stange	1:16.909	106.5
Sports Classics	JB 700	1:12.071	119.25
Sports Classics	Mamba	1:12.372	117
Sports Classics	Manana	1:23.183	98.25
Sports Classics	Monroe	1:11.172	122
Sports Classics	Peyote	1:24.751	98.25
Sports Classics	Pigalle	1:10.169	121.25
Sports Classics	Roosevelt	1:17.344	98.5
Sports Classics	Roosevelt Valor	1:16.406	98.5
Sports Classics	Stinger	1:13.239	112
Sports Classics	Stinger GT	1:12.972	112
Sports Classics	Stirling GT	1:06.166	112
Sports Classics	Tornado	1:26.353	98.25
Sports Classics	Tornado Cabrio	1:26.356	98.25
Sports Classics	Tornado Custom	1:24.417	98.25
Sports Classics	Tornado Mariachi	1:31.891	91.25
Sports Classics	Tornado Rat Rod	1:18.277	98.75
Sports Classics	Z-Type	1:09.368	126.25
Supers	811	1:03.930	129.25
Supers	Adder	1:04.964	124.75
Supers	Banshee 900R	1:04.565	131
Supers	Bullet	1:08.034	118.75
Supers	Cheetah	1:03.530	120.25
Supers	Entity XF	1:02.323	121.5
Supers	ETR1	1:01.728	120.75
Supers	FMJ	1:01.795	125
Supers	Infernus	1:02.929	117.75
Supers	Itali GTB	1:01.861	126.25
Supers	Itali GTB Custom	1:02.896	127
Supers	Nero	1:01.495	126.5
Supers	Nero Custom	1:01.061	127.25
Supers	Osiris	1:01.394	122
Supers	Penetrator	1:02.562	124
Supers	RE-7B	1:00.627	123.5
Supers	Reaper	1:02.716	121.75
Supers	Rocket Voltic	1:03.144	106
Supers	Sultan RS	1:04.364	117.25
Supers	T20	1:01.287	122.25
Supers	Tempesta	1:00.803	121.25
Supers	Turismo R	1:01.998	121.75
Supers	Tyrus	1:02.228	123.5
Supers	Vacca	1:03.964	120.25
Supers	Voltic	1:05.798	106
Supers	X80 Proto	1:01.175	127.5
Supers	Zentorno	1:00.960	122
'''

In [70]:
for line in data.splitlines():
    vdata = line.split('\t')
    if vdata[2]:
        vv = get_object_by_string(vdata[1], Vehicle, "name", score_cutoff=95)
        if not vv:
            print(".............................NOT FOUND:", vdata[1])
            continue
        millis = lt2millis(vdata[2])
        vv.cc_laptime_millis = millis
        vv.topspeed_kmh = float(vdata[3]) * 1.60934
        if (vdata[1] != vv.name):
            print(vdata[1], vv.name, vv.pk, vv.cc_laptime_millis, vv.topspeed_kmh)
        else:
            vv.save()

.............................NOT FOUND: Avarus
.............................NOT FOUND: Chimera
.............................NOT FOUND: Daemon (Lost)
.............................NOT FOUND: Defiler
.............................NOT FOUND: Esskey
.............................NOT FOUND: Faggio Mod
.............................NOT FOUND: Faggio Sport
.............................NOT FOUND: Manchez
.............................NOT FOUND: Nightblade
.............................NOT FOUND: Rat Bike
.............................NOT FOUND: Sanctus
.............................NOT FOUND: Vortex
.............................NOT FOUND: Wolfsbane
.............................NOT FOUND: Zombie Bobber
.............................NOT FOUND: Zombie Chopper
.............................NOT FOUND: 9F Cabrio
.............................NOT FOUND: Rocket Voltic


In [44]:
get_object_by_string("Raptor", Vehicle, "name", score_cutoff=.99)

<Vehicle: Penetrator (Super)>

In [61]:
v = Vehicle.objects.get(name="Raptor")

In [64]:
v.classes.all()

<QuerySet [<VehicleClass: Sport>]>

In [58]:
from vehicles.models import VehicleClass
vc  = VehicleClass.objects.get(name="Sport")

In [60]:
v.save()